In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
!pip install neuralprophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 26.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.wh

In [25]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [ ]:


path = '/content/'
data = pd.read_csv(path + 'historical_weather.csv')
first = data.groupby('city_id')
for i in range(1):
    first_group = first.get_group(list(first.groups.keys())[0])
    first_group.index = pd.to_datetime(first_group['date'])
    first_group = first_group.ffill()
    first_group = first_group.fillna(0)
    # print(first_group)
    print(first_group.apply(pd.isnull).sum())

city_id               0
date                  0
avg_temp_c            0
min_temp_c            0
max_temp_c            0
precipitation_mm      0
snow_depth_mm         0
avg_wind_dir_deg      0
avg_wind_speed_kmh    0
dtype: int64


In [133]:
# from neuralprophet import NeuralProphet
# import pandas as pd
# import numpy as np
# from sklearn.metrics import mean_squared_error


# path = '/content/'
# data = pd.read_csv(path + 'historical_weather.csv')

# first = data.groupby('city_id')
# first_group = first.get_group(list(first.groups.keys())[0])
# first_group['date'] = pd.to_datetime(first_group['date'])
# first_group = first_group.ffill().fillna(0)

def feature_pred(first_group,feature,day = '2018-12-25'):
    global regressors
    print(0)
    print(feature)
    print(0)
    # day_dt = pd.to_datetime(day)
    # start_date = day_dt - pd.DateOffset(days=7)
    # test_data = first_group[(first_group['ds'] > start_date) & (first_group['ds'] <= day_dt)][['ds', feature]]
    # test_data.columns = ['ds', 'y']
    # train_data = first_group[first_group['ds'] <= start_date][['ds', feature]]
    # train_data.columns = ['ds', 'y']

    train_data = first_group[first_group['ds'] < pd.to_datetime(day)][['ds', feature]]
    train_data.columns = ['ds', 'y']

    m = NeuralProphet()
    model = m.fit(train_data, freq='D', epochs=40)

    # Make predictions on the test data
    future = m.make_future_dataframe(train_data, periods=7)
    forecast = m.predict(future)
    # to_get_rmse=forecast[['ds', 'yhat1']].head(7)
    forecast_values = forecast[['ds', 'yhat1']].tail(7)



    # merged_data = pd.merge(test_data, to_get_rmse, on='ds', how='inner')
    # rmse = np.sqrt(mean_squared_error(merged_data['y'], merged_data['yhat1']))
    # print(rmse)

    # if rmse<5:
    return forecast_values['yhat1']
    # else:
    #   regressors=[col for col in regressors if col not in [feature]]
    #   return 0

In [134]:
from neuralprophet import NeuralProphet
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
day = '2018-12-25'
path = '/content/'
data = pd.read_csv(path + 'historical_weather.csv')
data=data.drop(columns='avg_wind_dir_deg')
regressors=[]
def drop_high_nan_columns(df, threshold=0.1):
    nan_percentage = df.isna().mean()
    columns_to_drop = nan_percentage[nan_percentage > threshold].index
    columns_to_drop = [col for col in columns_to_drop if col not in ['snow_depth_mm','avg_temp_c']]

    snow_nan_percentage = df['snow_depth_mm'].isna().mean()
    if snow_nan_percentage > 0.98:
        columns_to_drop.append('snow_depth_mm')
    df = df.ffill().fillna(0)
    return df.drop(columns=columns_to_drop)

rmsec=[]
# Group data by city_id
first = data.groupby('city_id')
for i in range(10):
    first_group = first.get_group(list(first.groups.keys())[i])
    first_group['date'] = pd.to_datetime(first_group['date'])
    first_group = drop_high_nan_columns(first_group)
    regressors = [col for col in first_group.columns if col not in ['date', 'avg_temp_c','city_id']]

    test_data = first_group[first_group['date'] >= pd.to_datetime(day)]
    train_data=first_group[first_group['date'] < pd.to_datetime(day)]

    test_data=test_data[['date', 'avg_temp_c'] + regressors]
    test_data.columns = ['ds', 'y'] + regressors

    # Prepare train data
    train_data = train_data[['date', 'avg_temp_c'] + regressors]
    train_data.columns = ['ds', 'y'] + regressors

    # Creating a DataFrame for future regressors

    last_date = train_data['ds'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7, freq='D')

    regressors_df = pd.DataFrame({
        'ds': future_dates,
        'min_temp_c': feature_pred(train_data, 'min_temp_c') if 'min_temp_c' in regressors else 0,
        'max_temp_c': feature_pred(train_data, 'max_temp_c') if 'max_temp_c' in regressors else 0,
        'precipitation_mm': feature_pred(train_data, 'precipitation_mm') if 'precipitation_mm' in regressors else 0,
        # 'avg_wind_dir_deg': feature_pred(train_data, 'avg_wind_dir_deg') if 'avg_wind_dir_deg' in regressors else 0,
        'avg_wind_speed_kmh': feature_pred(train_data, 'avg_wind_speed_kmh') if 'avg_wind_speed_kmh' in regressors else 0,
        'snow_depth_mm': feature_pred(train_data, 'snow_depth_mm') if 'snow_depth_mm' in regressors else 0,
    })
    regressors_df.drop(columns=[col for col in regressors_df.columns if col not in regressors+['ds']])

    print(regressors_df)
    print(regressors)
    train_data = train_data[['ds', 'y'] + regressors]
    test_data= test_data[['ds', 'y'] + regressors]



    # Add 7 empty rows to the end of train_data to account for future periods



    m = NeuralProphet()
    for regressor in regressors:
        m = m.add_future_regressor(name=regressor)

    # Fit the model
    model = m.fit(train_data.dropna(), freq='D', epochs=100,continue_training=True)
    # print(regressors_df)
    # Make predictions on the test data
    future = m.make_future_dataframe(df=train_data, regressors_df=regressors_df, periods=7)

    # Make predictions
    forecast = m.predict(future)

    # Extract forecast values
    forecast_values = forecast[['ds', 'yhat1']].tail(7)

    # Merge actual test data with forecast values for comparison
    merged_data = pd.merge(test_data, forecast_values, on='ds', how='inner')
    rmse = np.sqrt(mean_squared_error(merged_data['y'], merged_data['yhat1']))
    rmsec.append(rmse)
    print(f"RMSE: {rmse}")
    print(forecast_values)


WARNING - (py.warnings._showwarnmsg) - <ipython-input-134-974c36f73149>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_group['date'] = pd.to_datetime(first_group['date'])

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_group['date'] = pd.to_datetime(first_group['date'])

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO:NP.df_utils:Major frequency D 

0
min_temp_c
0


Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (57) is too small than the required number                     for the learning rate finder (232). The results might not be optimal.


Finding best initial lr:   0%|          | 0/232 [00:00<?, ?it/s]

...

WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 85.714% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 85.714% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major 

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


0

KeyboardInterrupt: 

In [54]:
from neuralprophet import NeuralProphet
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# Define the regressors and day of interest
regressors = ['min_temp_c', 'max_temp_c', 'precipitation_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh','snow_depth_mm']
day = '2018-12-25'
path = '/content/'

# Load the data
data = pd.read_csv(path + 'historical_weather.csv')

# Group data by city_id
first = data.groupby('city_id')
first_group = first.get_group(list(first.groups.keys())[0])
first_group['date'] = pd.to_datetime(first_group['date'])
first_group = first_group.ffill().fillna(0)

# Prepare test data
test_data = first_group[first_group['date'] >= pd.to_datetime(day)][['date', 'avg_temp_c'] + regressors]
test_data.columns = ['ds', 'y'] + regressors

# Prepare train data
train_data = first_group[first_group['date'] < pd.to_datetime(day)][['date', 'avg_temp_c'] + regressors]
train_data.columns = ['ds', 'y'] + regressors

# Add 7 empty rows to the end of train_data to account for future periods
last_date = train_data['ds'].max()
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7, freq='D')

# Creating a DataFrame for future regressors
regressors_df = pd.DataFrame({
    'ds': future_dates,
    'min_temp_c': 0,
    'max_temp_c': 0,
    'precipitation_mm': 0,
    'avg_wind_dir_deg': 0,
    'avg_wind_speed_kmh': 0,
    'snow_depth_mm': 0,
})

# Initialize and configure the model
m = NeuralProphet()

# Add additional regressors
for regressor in regressors:
    m = m.add_future_regressor(name=regressor)

# Fit the model
model = m.fit(train_data.dropna(), freq='D', epochs=10)
print(regressors_df)
# Make predictions on the test data
future = m.make_future_dataframe(df=train_data, regressors_df=regressors_df, periods=7)

# Make predictions
forecast = m.predict(future)

# Extract forecast values
forecast_values = forecast[['ds', 'yhat1']].tail(7)

# Merge actual test data with forecast values for comparison
merged_data = pd.merge(test_data, forecast_values, on='ds', how='inner')
rmse = np.sqrt(mean_squared_error(merged_data['y'], merged_data['yhat1']))

print(f"RMSE: {rmse}")
print(forecast_values)


WARNING - (py.warnings._showwarnmsg) - <ipython-input-54-fc7604b92e49>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_group['date'] = pd.to_datetime(first_group['date'])

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_group['date'] = pd.to_datetime(first_group['date'])

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (NP.df_utils.check_dataframe) - Encountered future regressor with only unique values in training set across all IDs.Auto

Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (57) is too small than the required number                     for the learning rate finder (232). The results might not be optimal.


Finding best initial lr:   0%|          | 0/232 [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 85.714% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 85.714% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 85.714% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 85.714% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is

          ds  min_temp_c  max_temp_c  precipitation_mm  snow_depth_mm  \
0 2018-12-25           0           0                 0              0   
1 2018-12-26           0           0                 0              0   
2 2018-12-27           0           0                 0              0   
3 2018-12-28           0           0                 0              0   
4 2018-12-29           0           0                 0              0   
5 2018-12-30           0           0                 0              0   
6 2018-12-31           0           0                 0              0   

   avg_wind_dir_deg  avg_wind_speed_kmh  
0                 0                   0  
1                 0                   0  
2                 0                   0  
3                 0                   0  
4                 0                   0  
5                 0                   0  
6                 0                   0  


ValueError: Unexpected column snow_depth_mm in data

In [23]:
import pandas as pd

# Create an empty DataFrame
df = pd.DataFrame(columns=['Column1', 'Column2'])

# Append a row of values
new_row = pd.DataFrame({'Column1': [1], 'Column2': [2]})
df = pd.concat([df, new_row], ignore_index=True)

In [135]:
print(rmsec)

[2.2715754714352108, 1.110549730770848, 1.124396124314359, 3.5648530375519045]
